# Лабораторная работа №1

Димитриев Анатолий ИДБ-18-09


In [22]:
import json
import csv
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta


Задание 1.

In [4]:
def listToDict(lst:list):

    return { val: [ k for k in range(i, len(lst)) if (val == lst[k]) ] for i, val in enumerate(lst) if i == (lst.index(val))}

listToDict(['a','b','c','d','e','a','a','b','c'])


{'a': [0, 5, 6], 'b': [1, 7], 'c': [2, 8], 'd': [3], 'e': [4]}

Задание 2.

In [12]:
def jaccardSimiliarity(setA:set, setB:set):
    countInter = 0
    for item in setA:
         if (item in setB):
             countInter+=1
    return countInter / (len(setA) + len(setB) - countInter)

print(jaccardSimiliarity({1,2,3,5},{1,4,3,5}))
print(jaccardSimiliarity({"Hello","World","Python"}, {"Hello","Python","Lab1"}))


0.6
0.5


Задание 4.

In [21]:
with open("salesJson.json") as sales:
    salesList = json.loads(sales.read())
sales.close()

with open("sales.csv", "w", newline="") as file:
    columns = ["item", "country", "year", "sales"]
    writer = csv.DictWriter(file, fieldnames=columns)
    writer.writeheader()

    for diction in salesList:
        for country, value in diction['sales_by_country'].items():
            for year in value:
                writer.writerow({"item":diction['item'],"country":country,"year":year,"sales":value[year]})


Задание 5.

In [23]:
def generatorDate(beginDate: date, endDate: date):
    while beginDate <= endDate:
        yield beginDate.strftime("%d/%m/%Y")
        beginDate += timedelta(days=1)

currencies = [
    "Доллар США",
    "Евро",
    "Индийская рупия",
    "Украинская гривна"
]

idResp = requests.get("http://www.cbr.ru/scripts/XML_val.asp?d=0")
soupParse = BeautifulSoup(idResp.content, 'xml')
currenciesId = {currency: soupParse.find("Name", text=currency).parent["ID"] for currency in currencies}


with open("currencies.csv",'w',newline="") as currencyFile:
    columns = ["Дата"] + currencies
    writer = csv.DictWriter(currencyFile, columns)
    writer.writeheader()

    for genDate in generatorDate(date(2020, 3, 1), date(2020, 7, 1)):
        currencies_price_response = requests.get(f"http://www.cbr.ru/scripts/XML_daily.asp?date_req={genDate}")
        mapRow = {columns[0]:genDate}
        soupParse = BeautifulSoup(currencies_price_response.content, 'xml')

        for currency, currencyId in currenciesId.items():
            value = float(soupParse.find(ID=currencyId).find("Value").get_text().replace(',','.'))
            nominal = int( soupParse.find(ID=currencyId).find("Nominal").get_text())
            course = round(value / nominal, 5)
            mapRow[currency] = course
        writer.writerow(mapRow)
        

    

